# Populating Book Crossing Dataset into Database

This notebook aims at loading the [Book Crossing](https://grouplens.org/datasets/book-crossing/) dataset into the `recsys` database. 

## Imports

In [1]:
import sys, os 
import numpy as np 
import pandas as pd 
from sqlalchemy import types
lib_path = './../Sources'
if (lib_path not in sys.path):
    sys.path.append(lib_path) #src directory
import lpsrec.database as db

## Getting Database Connection

In [13]:
username = 'postgres'
password = 'admin'
dbname = 'RecSys'
hostname = 'localhost:5432'
conn = db.get_database_connection(username, password, hostname, dbname)

In [15]:
conn.execute('commit')

## Loading files into pandas dataframes

In [3]:
dataset_path = './../Datasets/BookCrossing/Standard_raw/'
os.listdir(dataset_path)

['BX-Book-Ratings.csv', 'BX-Books.csv', 'BX-Users.csv', 'desktop.ini']

In [4]:
df_ratings = pd.read_csv(os.path.join(dataset_path, 'BX-Book-Ratings.csv'), sep=';')
print ("Number of ratings: ", df_ratings.shape[0])
print ("Number of users who have ratings: ", df_ratings['User-ID'].unique().shape[0])
print ("Number of items who have ratings: ", df_ratings['ISBN'].unique().shape[0])
df_ratings.head()

Number of ratings:  1149780
Number of users who have ratings:  105283
Number of items who have ratings:  340557


,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [5]:
df_items = pd.read_csv(os.path.join(dataset_path, 'BX-Books.csv'), sep=';')
print ("Number of items: ", df_items['ISBN'].unique().shape[0])
df_items.head()

Number of items:  271378


c:\anaconda\envs\rec_env\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [6]:
df_users = pd.read_csv(os.path.join(dataset_path, 'BX-Users.csv'), sep=';')
print ("Number of users: ", df_users['User-ID'].unique().shape[0])
df_users.head()

Number of users:  278858


,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [7]:
sql_str = "select * from datasets.dataset where version = 'BOOKX'"
df_dataset_info = pd.read_sql(con=conn, sql=sql_str)
df_dataset_info.head()

,id_dataset,name,url,registers,size,id_cluster,version,id_utility
0,7,Book Crossing,https://grouplens.org/datasets/book-crossing/,None,None,3,BOOKX,3


## Inserting Users 
Checking users who have ratings but don't have IDs on df_users

In [ ]:
set(df_ratings['User-ID'])-set(df_users['User-ID'])

Since the set is empty, all users from ratings have their corresponding register in the `BX-Users.csv` file

In [ ]:
df_datasets_user = pd.DataFrame(columns=["id_dataset", "id_user_dataset"])
df_datasets_user['id_user_dataset'] = np.unique(np.append(df_users['User-ID'].unique(), df_ratings['User-ID'].unique()))
df_datasets_user['id_dataset'] = np.repeat(df_dataset_info['id_dataset'][0], df_datasets_user.shape[0])
df_datasets_user.tail()

Registering users on `datasets.user`

In [ ]:
%%time
df_types = {"id_dataset": types.INTEGER(), "id_user_dataset": types.INTEGER()}
conn.execute("delete from datasets.user_info where id_user in (select id_user from datasets.user where id_dataset = {})" .format(df_dataset_info['id_dataset'][0]))
conn.execute("delete from datasets.user where id_user in (select id_user from datasets.user where id_dataset = {})" .format(df_dataset_info['id_dataset'][0]))
df_datasets_user.to_sql(con=conn, schema='datasets', if_exists='append', name='user', index=False, dtype=df_types)

## Inserting Items
Checking items who have ratings but don't have IDs on df_items

In [8]:
set(df_ratings['ISBN'])-set(df_items['ISBN'])

{'0785316531',
 '0840712898',
 '3548363008',
 '3980312968',
 '0285629298',
 '3442430917',
 '3453092996',
 '8434581477',
 '0130202010',
 '0340610557',
 '0140471405',
 '0751530808',
 '0671808826',
 '0277300594',
 '0440843502',
 '0224050575',
 '1857994159',
 '0340241748',
 '882002438186I',
 '0340750014',
 '3930419726',
 '9510149519',
 '0743415558',
 '8845916332',
 '8845419258',
 '8471769123',
 '8804522534',
 '8440648650',
 '9722215876',
 '8804517891',
 '0785301712',
 '8420601837',
 '782253005278',
 '8432046930',
 '9722017926',
 '9782253066019',
 '3442235006',
 '0440183057495',
 '842268666X',
 '0352325313',
 '1561566004',
 '9022915891',
 '0330261487',
 '04484407X',
 '0523480466',
 '8481305057',
 '0333741463',
 '9724726347',
 '3872942670',
 '8481710024',
 '186049840X',
 '1568650590',
 '3442238110',
 'N0385292244',
 '0207146845',
 '033025409X',
 '0099230917',
 '8432040711',
 '84932229652',
 '8817153567',
 '3596108535',
 '074726323X',
 '0747404402',
 '2841720942',
 '7505940872',
 '1840119411'

We can see that **multiple items have ratings but they don't have their respective register on `BX-Books.csv`**. These items will be inserted in the `datasets.item` table anyhow.

In [9]:
df_datasets_item = pd.DataFrame(columns=["id_dataset", "id_item_dataset"])
df_datasets_item['id_item_dataset'] = np.unique(np.append(df_items['ISBN'].unique(), df_ratings['ISBN'].unique()))
df_datasets_item['id_dataset'] = np.repeat(df_dataset_info['id_dataset'][0], df_datasets_item.shape[0])
df_datasets_item.tail()

,id_dataset,id_item_dataset
341760,7,cn113107
341761,7,ooo7156103
341762,7,Խcrosoft
341763,7,�3499128624
341764,7,�423350229


Registering items on `datasets.item`

In [17]:
%%time
df_types = {"id_dataset": types.INTEGER(), "id_item_dataset": types.INTEGER()}
# print ("Aqui 1")
# conn.execute("delete from datasets.item_info where id_item in (select id_item from datasets.item where id_dataset = {})" .format(df_dataset_info['id_dataset'][0]))
# print ("Aqui 2")
# conn.execute('commit')
# print ("Aqui 3")
# conn.execute("delete from datasets.item where id_item in (select id_item from datasets.item where id_dataset = {})" .format(df_dataset_info['id_dataset'][0]))
# print ("Inserting data into database...")
df_datasets_item.to_sql(con=conn, schema='datasets', name='item', if_exists='append', index=False, dtype=df_types)

Wall time: 33.8 s


In [18]:
df_datasets_item.shape

(341765, 2)

## Inserting Ratings

In [22]:
df_ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [23]:
df_ratings.to_sql(con=conn, schema='transistory', if_exists='replace', name='bkx_data', index=False, dtype={'User-ID': types.VARCHAR(100), 'ISBN': types.VARCHAR(100), 'Book-Rating': types.INTEGER()  })

In [27]:
sql_str = """
insert into datasets.feedback 
(id_user, id_item, value, id_feedback_type)
select us.id_user, it.id_item, tran."Book-Rating", ft.id_feedback_type  
from transistory.bkx_data tran 
inner join datasets."user" us on us.id_user_dataset = tran."User-ID" and us.id_dataset = """ + str(df_dataset_info['id_dataset'][0]) + """
inner join datasets.item it on it.id_item_dataset = tran."ISBN" and it.id_dataset = """ + str(df_dataset_info['id_dataset'][0]) + """
inner join datasets.feedback_type ft on ft.info_type = 'explicit'
where it.id_item_dataset != 'null'"""
conn.execute(sql_str)
